In [0]:
df_silver = spark.table("uc_athlete_data.silver.strava_activities").select("athlete_id","start_date","ingestion_timestamp")
display(df_silver)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
from pyspark.sql.functions import lit

# Definindo a window
w_last_activity_date = Window.partitionBy("athlete_id").orderBy(col("start_date").desc())
w_first_ingestion_date = Window.partitionBy("athlete_id").orderBy(col("ingestion_timestamp").asc())

# Aplicando row_number
df_last_activity = df_silver.withColumn("row_last_actitivity", row_number().over(w_last_activity_date))
df_first_ingestion = df_silver.withColumn("row_first_ingestion", row_number().over(w_first_ingestion_date))

#Aplica Filtro Ultima atividade e primeira ingestão
df_last_activity = df_last_activity.filter(col("row_last_actitivity") == 1).select("athlete_id","start_date")
df_first_ingestion = df_first_ingestion.filter(col("row_first_ingestion") == 1).select("athlete_id","ingestion_timestamp")


display(df_last_activity)
display(df_first_ingestion)

# Mantendo apenas a 1ª linha de cada grupo
#df = df_ranked.filter(col("row_num") == 1)#.drop("row_num")
#display(df)

In [0]:


df_tb_controle = df_last_activity.join(df_first_ingestion, "athlete_id", "inner").withColumn("full_carga", lit("N"))  
display(df_tb_controle)

In [0]:
from pyspark.sql.functions import unix_timestamp, col

def convert_to_unix(df, column_name, new_column_name=None):
    """
    Converte uma coluna de timestamp para Unix timestamp (segundos).
    """
    if not new_column_name:
        new_column_name = f"{column_name}_unix"
    return df.withColumn(new_column_name, unix_timestamp(col(column_name)))

df_tb_controle = convert_to_unix(df_tb_controle, "start_date")
df_tb_controle = convert_to_unix(df_tb_controle, "ingestion_timestamp")
display(df_tb_controle)


In [0]:
path = "abfss://silver@adlsathlete.dfs.core.windows.net/data-move/"
file_name = "abfss://silver@adlsathlete.dfs.core.windows.net/strava/control_load_table.csv"

df_tb_controle.repartition(1).write.options(header='True', delimiter=',') \
        .option("escapeQuotes", "false") \
        .option("enconding", "UTF-8") \
        .csv(path)

arquivos = dbutils.fs.ls(path)

for arquivo in arquivos:
    if arquivo.name.startswith("part-"):
        arquivo_part = arquivo.path
        dbutils.fs.mv(arquivo_part, file_name)

dbutils.fs.rm(path, True)
 

In [0]:
%sql
drop table uc_athlete_data.silver.control_load_table

In [0]:
df_tb_controle.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("uc_athlete_data.silver.control_load_table")


In [0]:
%sql
select * from uc_athlete_data.silver.control_load_table